In [1]:
import time
from tqdm import tqdm

In [2]:
import tensorflow as tf

2024-04-09 02:37:13.340950: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 02:37:13.911032: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from tensorflow.keras.models import load_model

In [4]:
import numpy as np

In [5]:
model = load_model('surogate.h5')

2024-04-09 02:37:15.517130: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-04-09 02:37:15.517695: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [6]:
# 特定频率
def loss_freq(s_para):
    # 能量
    E = tf.square(s_para)
    E11 = E[:,0] + E[:,1]
    E21 = E[:,2] + E[:,3]
    E31 = E[:,4] + E[:,5]
    E41 = E[:,6] + E[:,7]
    P21 = tf.math.atan2(s_para[:,3], s_para[:,2])
    P31 = tf.math.atan2(s_para[:,5], s_para[:,4])

    # 压制
    loss1 = E11 - E21 - E31 + E41
    # 比例
    loss2 = tf.abs(E21 / (E31 + E21) - 0.7)
    # phase
    loss3 = tf.abs(P21 - P31 - np.pi / 4)
    # 能量守恒
    loss4 = tf.abs(tf.reduce_sum(E, axis=1) - 1)
    
    return loss1 + loss2 + loss3 + loss4

In [7]:
def dgn_obj(s1, s2, s3):
    # var = tf.reduce_sum(tf.nn.moments(tf.stack([s1, s2, s3], axis=1), [1])[1][:, 2: 6], axis=-1)
    # return tf.reduce_max([loss_freq(s1), loss_freq(s2), loss_freq(s3)], axis=0)
    return loss_freq(s1) + loss_freq(s2) + loss_freq(s3)

In [8]:
# 参数数量，迭代次数
num_nodes = 200
num_node_epochs = 3000

In [9]:
# 物理边界
mmin = np.array([1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4])
mmax = np.array([5, 5, 5, 5, 5, 5, 5, 5, 100, 100, 100, 100])

In [10]:
# 物理边界检查函数
def check(GeomData):
    inva_place1 = tf.where(tf.logical_or(GeomData[:,:8] < 1, GeomData[:,:8] > 5))
    GeomData = tf.tensor_scatter_nd_update(GeomData, [inva_place1], [np.random.uniform(mmin[inva_place1[:,1]], mmax[inva_place1[:,1]], (inva_place1.shape[0]))])
    
    inva_place2 = tf.where(tf.logical_or(GeomData[:,8:] < 4, GeomData[:,8:] > 100)) + [0, 8]
    GeomData = tf.tensor_scatter_nd_update(GeomData, [inva_place2], [np.random.uniform(mmin[inva_place2[:,1]], mmax[inva_place2[:,1]], (inva_place2.shape[0]))])
    
    return tf.Variable(GeomData)

In [11]:
# 初始数据
geom = tf.Variable(np.random.uniform(mmin, mmax, (num_nodes, 12)), dtype=tf.float32)
freq1 = np.ones((num_nodes, 1), dtype=np.float32) * 2.4
freq2 = np.ones((num_nodes, 1), dtype=np.float32) * 2.5
freq3 = np.ones((num_nodes, 1), dtype=np.float32) * 2.6

In [12]:
bestLoss = 10
bestGeom = 0

In [13]:
# 创建优化器
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.01)

In [14]:
logs = []

In [15]:
# 优化过程
t0 = time.time()
for i in tqdm(range(num_node_epochs)):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(geom)
        y_pred1 = model(tf.concat([freq1, geom], axis=1))
        y_pred2 = model(tf.concat([freq2, geom], axis=1))
        y_pred3 = model(tf.concat([freq3, geom], axis=1))
        loss = dgn_obj(y_pred1, y_pred2, y_pred3)
    minLoss = tf.reduce_min(loss).numpy()
    minIndex = tf.argmin(loss).numpy()
    minGeom = geom[minIndex].numpy()
    grads = tape.gradient(loss, geom)
    opt.apply_gradients(grads_and_vars=zip([grads], [geom]))
    geom = check(geom)
    if minLoss < bestLoss:
        bestLoss = minLoss
        bestGeom = minGeom
        # print(minIndex)
        # print(i, bestLoss)
        # print(" ".join(map(str, bestGeom)))
        # print()
    logs.append({'epoch': i, 'time': time.time() - t0, 'acc': bestLoss})

100%|███████████████████████████████████████| 3000/3000 [02:22<00:00, 21.08it/s]


In [16]:
logs

[{'epoch': 0, 'time': 0.4713003635406494, 'acc': -0.3074453},
 {'epoch': 1, 'time': 0.5965917110443115, 'acc': -0.35711646},
 {'epoch': 2, 'time': 0.645916223526001, 'acc': -0.39316165},
 {'epoch': 3, 'time': 0.6896748542785645, 'acc': -0.42360598},
 {'epoch': 4, 'time': 0.7331750392913818, 'acc': -0.45036954},
 {'epoch': 5, 'time': 0.7813019752502441, 'acc': -0.47555673},
 {'epoch': 6, 'time': 0.8249590396881104, 'acc': -0.49946016},
 {'epoch': 7, 'time': 0.8684878349304199, 'acc': -0.5221923},
 {'epoch': 8, 'time': 0.9162931442260742, 'acc': -0.5442482},
 {'epoch': 9, 'time': 0.9599452018737793, 'acc': -0.56533873},
 {'epoch': 10, 'time': 1.0035512447357178, 'acc': -0.5843681},
 {'epoch': 11, 'time': 1.0516691207885742, 'acc': -0.60343283},
 {'epoch': 12, 'time': 1.095299482345581, 'acc': -0.6226145},
 {'epoch': 13, 'time': 1.1388704776763916, 'acc': -0.6410409},
 {'epoch': 14, 'time': 1.187098503112793, 'acc': -0.6585706},
 {'epoch': 15, 'time': 1.2308003902435303, 'acc': -0.6761930

In [17]:
for log in logs:
    print(log['time'], log['acc'])

0.4713003635406494 -0.3074453
0.5965917110443115 -0.35711646
0.645916223526001 -0.39316165
0.6896748542785645 -0.42360598
0.7331750392913818 -0.45036954
0.7813019752502441 -0.47555673
0.8249590396881104 -0.49946016
0.8684878349304199 -0.5221923
0.9162931442260742 -0.5442482
0.9599452018737793 -0.56533873
1.0035512447357178 -0.5843681
1.0516691207885742 -0.60343283
1.095299482345581 -0.6226145
1.1388704776763916 -0.6410409
1.187098503112793 -0.6585706
1.2308003902435303 -0.67619306
1.2743339538574219 -0.6924184
1.3229100704193115 -0.7085418
1.3665618896484375 -0.724577
1.4101893901824951 -0.7406591
1.4584674835205078 -0.75679106
1.5024051666259766 -0.7732197
1.5497472286224365 -0.78966445
1.5983760356903076 -0.80604154
1.641925573348999 -0.822583
1.694056749343872 -0.8395974
1.7411882877349854 -0.8569738
1.7883892059326172 -0.8744424
1.832096815109253 -0.8921545
1.8825976848602295 -0.91015565
1.9232914447784424 -0.9285342
1.9666032791137695 -0.9469561
2.0138630867004395 -0.96565306
2.05